In [1]:
import pandas as pd
import geopandas as gpd
import os


Open Area Deprivation Index for Wisconsin (found [here](https://www.neighborhoodatlas.medicine.wisc.edu/)). 

From the Neighborhood Atlas site:
<br>"The Area Deprivation Index (ADI) is based on a measure created by the Health Resources & Services Administration (HRSA) over two decades ago for primarily county-level use, but refined, adapted, and validated to the Census block group/neighborhood level by Amy Kind, MD, PhD and her research team at the University of Wisconsin-Madison. It allows for rankings of neighborhoods by socioeconomic status disadvantage in a region of interest (e.g. at the state or national level). It includes factors for the theoretical domains of income, education, employment, and housing quality. It can be used to inform health delivery and policy, especially for the most disadvantaged neighborhood groups."

In [2]:
wi_adi = pd.read_csv(os.path.join("../../wi_bg_v1.5.txt"),delimiter=",")
wi_adi.head()

,GISJOIN,FIPS,ADI_STATERNK,ADI_NATRANK
0,G55000109501001,550019501001,5,47
1,G55000109501002,550019501002,4,44
2,G55000109501003,550019501003,3,38
3,G55000109502011,550019502011,8,67
4,G55000109502021,550019502021,6,56
